<a href="https://colab.research.google.com/github/yossy-goddog/Prog2kakushin/blob/main/work2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('numpyを使って遊び大全のゲームを作りたい')

In [ ]:
import numpy as np
import sys
import random
import copy

field=np.zeros((6, 7), dtype=int)
(x, y)=field.shape
def player_computer(field,number):
    if (number <= 0) or (y < number):
        print('エラー:選んだ数字はこの枠内からはみ出します')
        sys.exit()
    for i in range(len(field)-1, -1, -1):
        point=field[(i, number - 1)]
        if point==0:
            return (i, number - 1)
    print('エラー:そこの列は空いてません')
    sys.exit()
def field_saishin_player(field,xp,yp):
    if field[(xp,yp)] != 0:
        print('エラー:そこは0ではありません')
        sys.exit()
    field[xp,yp]=1
    return field
def field_saishin_computer(field,xp,yp):
    if field[(xp,yp)]!=0:
        print('エラー:そこは0ではありません')
        sys.exit()
    field[xp,yp]=-1
    return field
def computer_choose_number(field):
    number_list=[]
    transpose_field=np.transpose(field)
    for i in range(len(transpose_field)):
        if(0 in transpose_field[i]):
            number_list.append(i+1)
    r=random.randrange(0, len(number_list))
    return number_list[r]
def check_reach_row(field):
    #rowにリーチがかかっているか確認
    p1_list=[]
    copy_field=copy.copy(field)
    # -1を削除
    copy_field[field<0]=0
    for i in range(0,len(field)):
        value=0
        for j in range(0,len(field[0])):
            value=value*copy_field[(i, j)]+copy_field[(i, j)]
            copy_field[(i, j)]=value
            #value==2
            if(value==2 and (j+1)<len(field[0]) and (j+2)<len(field[0])):
                if((field[(i, j+1)]==0) and (field[(i, j+2)]==1)):
                    p1_list.append((i, j+1))
            if(value==2 and (j-2)>=0) and ((j-3)>=0):
                if((field[(i, j-2)]==0) and (field[(i, j-3)]==1)):
                    p1_list.append((i, j-2))
             #value==3
            if(value==3 and (j+1)<len(field[0])):
               if(field[(i, j+1)]==0 and (i, j+1) not in p1_list):
                   p1_list.append((i, j+1))
            if(value==3 and (j-3)>=0):
               if(field[(i, j-3)]==0 and (i, j-3) not in p1_list):
                   p1_list.append((i, j-3))
    return p1_list
def check_reach_col(field):
    # colにリーチがかかっていないか確認
    transpose_field=np.transpose(field)
    p2_list=check_reach_row(transpose_field)
    new_p_list=[]
    for i in range(0,len(p2_list)):
        v1,v2=p2_list[i]
        new_p_list.append((v2, v1))
    return new_p_list
def computer_choose_number2(field):
    p1_list=check_reach_row(field)
    p2_list=check_reach_col(field)
    p_list=list(set(p1_list) | set(p2_list))
    if(len(p_list)!=0):
      number_list=[]
      for i in p_list:
          v1,v2=i
          if((v2+1) not in number_list):
                number_list.append(v2+1)
      r=random.randrange(0,len(number_list))
      return number_list[r]
    else:
      return computer_choose_number(field)

def victory_row(field):
    field_player=copy.copy(field)
    field_computer=copy.copy(field)
    # -1をなくす
    field_player[field < 0]=0
    # 1をなくす
    field_computer[field > 0]=0
    for i in range(len(field[:,0])):
        value_p=0
        value_c=0
        for j in range(len(field[0])):
            value_p=value_p*field_player[(i, j)]+field_player[(i, j)]
            value_c=value_c*np.absolute(field_computer[(i, j)])+field_computer[(i, j)]
            field_player[(i, j)]=value_p
            field_computer[(i, j)]=value_c
    # 4が含まれているか
    check_value_player=False
    count=0
    while check_value_player==False and count < len(field):
        check_value_player=4 in field_player[count]
        count +=1
   # -4が含まれているか
    check_value_computer=False
    count=0
    while check_value_computer==False and count < len(field):
         check_value_computer=-4 in field_computer[count]
         count +=1
    return check_value_player, check_value_computer
def victory_col(field):
    transpose_field=np.transpose(field)
    check_value_player,check_value_computer=victory_row(transpose_field)
    return check_value_player, check_value_computer
def victory_slash(field):
    shift_field = np.zeros(field.shape, dtype=int)
    for i in range(len(shift_field)):
        shift_field[i]=np.roll(field[i], -i)
    check_value_player,check_value_computer=victory_col(shift_field)
    return check_value_player, check_value_computer
def victory_backslash(field):
    shift_field=np.zeros(field.shape, dtype=int)
    for i in range(len(shift_field)):
        shift_field[i]=np.roll(field[i], i)
    check_value_player, check_value_computer=victory_col(shift_field)
    return check_value_player, check_value_computer
def victory(field):
    result_player=False
    result_computer=False
    r_player, r_computer=victory_row(field)
    c_player, c_computer=victory_col(field)
    s_player, s_computer=victory_slash(field)
    b_palyer, b_computer=victory_backslash(field)
    if(r_player or c_player or s_player or b_palyer):
        result_player=True
        print('You won.')
        return result_player, result_computer
    if(r_computer or c_computer or s_computer or b_computer):
        result_computer=True
        print('Computer won.')
        return result_player, result_computer
    return result_player, result_computer
def output(field):
    print('----------------------')
    print('|１|２|３|４|５|６|７|')
    print('----------------------')
    for i in range(0, len(field)):
        print('----------------------')
        for j in range(0, len(field[0])):
            if(field[(i, j)]==1):
                s = '○'
            elif(field[(i, j)]==-1):
                s = '×'
            else:
                s = '　'
            print('|{0}'.format(s), end='')
        print('|')
    print('----------------------')
result_player=False
result_computer=False
f=input('先攻、後攻どっちを選びますか: ')
while f=='先攻' and result_player==False and result_computer==False:
  print(f'1から{len(field[0])}の中で数字を一つ選んでください')
  select_number=int(input('select number: '))
  point=player_computer(field,select_number)
  field=field_saishin_player(field,point[0],point[1])
  output(field)
  result_player, result_computer=victory(field)
  if result_player:
     break
  print('computerの番です')
  select_number=computer_choose_number2(field)
  print(f'computerは{select_number}を選びました。')
  point=player_computer(field,select_number)
  field=field_saishin_computer(field,point[0],point[1])
  output(field)
  result_player, result_computer=victory(field)
  if result_computer:
     break
while f=='後攻' and result_player==False and result_computer==False:
  print('computerの番です')
  select_number=computer_choose_number2(field)
  print(f'computerは{select_number}を選びました。')
  point=player_computer(field,select_number)
  field=field_saishin_computer(field,point[0],point[1])
  output(field)
  result_player, result_computer=victory(field)
  if result_computer:
     break
  print(f'1から{len(field[0])}の中で数字を一つ選んでください')
  select_number=int(input('select number: '))
  point=player_computer(field,select_number)
  field=field_saishin_player(field,point[0],point[1])
  output(field)
  result_player, result_computer=victory(field)
  if result_player:
     break
if f != '後攻' and f != '先攻':
   print('エラー:先攻か後攻を選べていません')
   sys.exit()

先攻、後攻どっちを選びますか: 先攻
1から7の中で数字を一つ選んでください
select number: 1
----------------------
|１|２|３|４|５|６|７|
----------------------
----------------------
|　|　|　|　|　|　|　|
----------------------
|　|　|　|　|　|　|　|
----------------------
|　|　|　|　|　|　|　|
----------------------
|　|　|　|　|　|　|　|
----------------------
|　|　|　|　|　|　|　|
----------------------
|○|　|　|　|　|　|　|
----------------------
computerの番です
computerは7を選びました。
----------------------
|１|２|３|４|５|６|７|
----------------------
----------------------
|　|　|　|　|　|　|　|
----------------------
|　|　|　|　|　|　|　|
----------------------
|　|　|　|　|　|　|　|
----------------------
|　|　|　|　|　|　|　|
----------------------
|　|　|　|　|　|　|　|
----------------------
|○|　|　|　|　|　|×|
----------------------
1から7の中で数字を一つ選んでください
select number: 2
----------------------
|１|２|３|４|５|６|７|
----------------------
----------------------
|　|　|　|　|　|　|　|
----------------------
|　|　|　|　|　|　|　|
----------------------
|　|　|　|　|　|　|　|
----------------------
|　|　|　|　|　|　|　|
--------------------